# Model serialization

## Import model

In [ ]:
from __future__ import annotations

import json
from collections import abc
from difflib import get_close_matches
from typing import Any, Callable, Sequence, Union
from warnings import warn

import sympy as sp
from ampform.dynamics import BlattWeisskopfSquared
from ampform.kinematics.phasespace import Kallen
from ampform.sympy import argument, unevaluated
from attrs import asdict, frozen
from IPython.display import JSON, Markdown, Math
from sympy.core.symbol import Str
from sympy.physics.quantum.spin import Rotation as Wigner
from sympy.printing.latex import LatexPrinter

from ampform_dpd import (
    DefinedExpression,
    _create_coupling_symbol,  # noqa:PLC2701
)
from ampform_dpd.angles import formulate_scattering_angle
from ampform_dpd.decay import (
    FinalStateID,
    IsobarNode,
    Particle,
    State,
    StateID,
    ThreeBodyDecay,
    ThreeBodyDecayChain,
)
from ampform_dpd.dynamics import BuggBreitWigner, EnergyDependentWidth, FormFactor, P
from ampform_dpd.io import as_markdown_table, aslatex, simplify_latex_rendering

simplify_latex_rendering()

In [ ]:
with open("Lc2ppiK.json") as stream:
    MODEL_DEFINITION = json.load(stream)

In [ ]:
JSON(MODEL_DEFINITION)

## Construct `ThreeBodyDecay`

In [ ]:
def to_decay(
    model: dict, to_latex: Callable[[str], str] | None = None
) -> ThreeBodyDecay:
    initial_state = _get_initial_state(model)
    final_state = _get_final_state(model)
    return ThreeBodyDecay(
        states=_get_states(model),
        chains=sorted({
            to_decay_chain(chain, initial_state, final_state, to_latex)
            for chain in _get_decay_chains(model)
        }),
    )


def _get_decay_chains(model: dict) -> list[dict]:
    distribution_def = _get_distribution_def(model)
    return distribution_def["decay_description"]["chains"]


def _get_distribution_def(model: dict) -> dict:
    distribution_defs = MODEL_DEFINITION["distributions"]
    n_distributions = len(distribution_defs)
    if n_distributions == 0:
        msg = "The serialized model does not have any distributions"
        raise ValueError(msg)
    if n_distributions > 1:
        msg = f"There are {n_distributions} distributions, but expecting one only"
        warn(msg, category=UserWarning)
    return distribution_defs[0]


def to_decay_chain(
    chain_def: dict,
    initial_state: State,
    final_state: dict[FinalStateID, State],
    to_latex: Callable[[str], str] | None = None,
) -> ThreeBodyDecayChain:
    vertices = chain_def["vertices"]
    if to_latex is None:
        to_latex = lambda x: x  # noqa:E731
    resonance = Particle(
        name=chain_def["name"],
        latex=name_to_latex(chain_def["name"]),
        spin=chain_def["propagators"][0]["spin"],
        mass=0,
        width=0,
        parity=None,
    )
    return ThreeBodyDecayChain(
        decay=IsobarNode(
            parent=initial_state,
            child1=IsobarNode(
                parent=resonance,
                child1=final_state[vertices[1]["node"][0]],
                child2=final_state[vertices[1]["node"][1]],
            ),
            child2=final_state[vertices[0]["node"][1]],
        )
    )


def _get_states(model: dict) -> dict[StateID, State]:
    initial_state = _get_initial_state(model)
    final_state = _get_final_state(model)
    return {initial_state.index: initial_state, **final_state}


def _get_initial_state(model: dict) -> dict[StateID, State]:
    distribution_def = _get_distribution_def(model)
    decay_description = distribution_def["decay_description"]
    kinematics = decay_description["kinematics"]
    return dict_to_particle(kinematics["initial_state"])


def _get_final_state(model: dict) -> dict[StateID, State]:
    distribution_def = _get_distribution_def(model)
    decay_description = distribution_def["decay_description"]
    kinematics = decay_description["kinematics"]
    final_state_def = kinematics["final_state"]
    return {p["index"]: dict_to_particle(p) for p in final_state_def}


def dict_to_particle(dct: dict) -> State:
    return State(
        name=dct["name"],
        latex=name_to_latex(dct["name"]),
        mass=dct["mass"],
        width=0,
        spin=dct["spin"],
        parity=None,
        index=dct["index"],
    )

In [ ]:
def name_to_latex(name: str) -> str:
    latex = {
        "Lc": R"\Lambda_c^+",
        "pi": R"\pi^+",
        "K": "K^-",
        "p": "p",
    }.get(name)
    if latex is not None:
        return latex
    mass_str = name[1:].strip("(").strip(")")
    subsystem_letter = name[0]
    subsystem = {"D": "D", "K": "K", "L": R"\Lambda"}.get(subsystem_letter)
    if subsystem is None:
        return name
    return f"{subsystem}({mass_str})"


DECAY = to_decay(MODEL_DEFINITION, to_latex=name_to_latex)

In [ ]:
Math(aslatex(DECAY))

In [ ]:
Markdown(as_markdown_table(DECAY))

## Dynamics

:::{seealso} [RUB-EP1/amplitude-serialization#22](https://github.com/RUB-EP1/amplitude-serialization/issues/22)
:::

### Function look-up mechanism

In [ ]:
function_defs = MODEL_DEFINITION["functions"]
{f["type"] for f in function_defs}

In [ ]:
CHAIN_DEFINITIONS = _get_decay_chains(MODEL_DEFINITION)
CHAIN_DEFINITIONS[2]

In [ ]:
def get_function_definition(function_name: str, model: dict) -> dict:
    function_definitions = model["functions"]
    for function_def in function_definitions:
        if function_def["name"] == function_name:
            return function_def
    existing_names = {f["name"] for f in function_definitions}
    msg = f"Could not find function with name {function_name!r}."
    candidates = get_close_matches(function_name, existing_names)
    if candidates:
        msg += f" Did you mean any of these? {', '.join(sorted(candidates))}"
    raise KeyError(msg)

In [ ]:
get_function_definition("BlattWeiskopf", MODEL_DEFINITION)

<!-- cspell:ignore Weiskopf -->
### Vertices

#### Blatt-Weisskopf form factor

In [ ]:
get_function_definition("BlattWeisskopf_resonance_l1", MODEL_DEFINITION)

In [ ]:
get_function_definition("BlattWeisskopf_resonance_l2", MODEL_DEFINITION)

In [ ]:
def _node_to_mass(node_item: int | Sequence[int]) -> sp.Symbol:
    if isinstance(node_item, int):
        return sp.Symbol(f"m{node_item}", nonnegative=True)
    if (
        isinstance(node_item, abc.Sequence)
        and all(isinstance(i, int) for i in node_item)
        and len(node_item) == 2
    ):
        ij = "".join(str(i) for i in node_item)
        return sp.Symbol(f"s{ij}", nonnegative=True)
    msg = f"Cannot create mass symbol for node {node_item}"
    raise NotImplementedError(msg)


sp.Tuple(_node_to_mass([2, 3]), _node_to_mass(1))

In [ ]:
def _node_to_mandelstam(node: Topology) -> sp.Symbol:
    if all(isinstance(i, int) for i in node):
        return _node_to_mass(node)
    return _node_to_mass(0)


sp.Tuple(_node_to_mandelstam([2, 3]), _node_to_mandelstam([[2, 3], 1]))

In [ ]:
def formulate_form_factor(vertex: dict, model: dict) -> DefinedExpression:
    function_name = vertex.get("formfactor")
    if not function_name:
        return DefinedExpression()
    function_definition = get_function_definition(function_name, model)
    function_type = function_definition["type"]
    if function_type == "BlattWeisskopf":
        node = vertex["node"]
        s = _node_to_mandelstam(node)
        m1, m2 = (_node_to_mass(i) for i in node)
        if all(isinstance(i, int) for i in node):
            meson_radius = sp.Symbol(R"R_\mathrm{res}")
        else:
            initial_state = _get_initial_state(model)
            meson_radius = sp.Symbol(f"R_{{{initial_state.latex}}}")
        angular_momentum = int(function_definition["l"])
        return DefinedExpression(
            expression=FormFactor(s, m1, m2, angular_momentum, meson_radius),
            definitions={
                meson_radius: function_definition["radius"],
            },
        )
    msg = f"No form factor implementation for {function_name!r}"
    raise NotImplementedError(msg)


CHAIN_2 = CHAIN_DEFINITIONS[2]
Math(aslatex(formulate_form_factor(CHAIN_2["vertices"][0], MODEL_DEFINITION)))

In [ ]:
s, m1, m2, d, L, z = sp.symbols("s m1 m2 d L z")
exprs = [
    FormFactor(s, m1, m2, L, d),
    BlattWeisskopfSquared(z, L),
]
Math(aslatex({e: e.evaluate() for e in exprs}))

### Propagators

#### Breit-Wigner

In [ ]:
@unevaluated
class BreitWigner(sp.Expr):
    s: Any
    mass: Any
    width: Any
    m1: Any = 0
    m2: Any = 0
    angular_momentum: Any = 0
    meson_radius: Any = 1
    _latex_repr_ = R"\mathcal{{R}}^\mathrm{{BW}}_{{{angular_momentum}}}\left({s}\right)"  # noqa:RUF027

    def evaluate(self):
        width = self.energy_dependent_width()
        return BreitWigner(self.s, self.mass, width)

    def energy_dependent_width(self) -> sp.Expr:
        s, m0, Γ0, m1, m2, L, d = self.args
        if L == 0 and m1 == 0 and m2 == 0:
            return Γ0
        return EnergyDependentWidth(s, m0, Γ0, m1, m2, L, d)

    def _latex_repr_(self, printer: LatexPrinter, *args) -> str:
        s = printer._print(self.s)
        function_symbol = R"\mathcal{R}^\mathrm{BW}"
        arg = Rf"\left({s}\right)"
        if self.angular_momentum == 0:
            if self.m1 == 0 and self.m2 == 0:
                width = printer._print(self.energy_dependent_width())
                arg = Rf"\left({s}; {width}\right)"
            return Rf"{function_symbol}{arg}"
        L = printer._print(self.angular_momentum)
        return Rf"{function_symbol}_{{{L}}}{arg}"


x, y, z = sp.symbols("x:z")
s, m0, Γ0, m1, m2, L, d = sp.symbols("s m0 Gamma0 m1 m2 L R")
exprs = [
    BreitWigner(s, m0, Γ0, m1, m2, L, d),
    BreitWigner(s, m0, Γ0),
    EnergyDependentWidth(s, m0, Γ0, m1, m2, L, d),
    FormFactor(s, m1, m2, L, d),
    P(s, m1, m2),
    Kallen(x, y, z),
]
Math(aslatex({e: e.doit(deep=False) for e in exprs}))

In [ ]:
CHAIN_DEFINITIONS[20]

In [ ]:
get_function_definition("K892_BW", MODEL_DEFINITION)

In [ ]:
def _formulate_breit_wigner(
    propagator: dict, function_definition: dict, resonance: str, **kwargs
) -> DefinedExpression:
    node = propagator["node"]
    i, j = node
    s = _node_to_mandelstam(node)
    mass = sp.Symbol(f"m_{{{resonance}}}")
    width = sp.Symbol(Rf"\Gamma_{{{resonance}}}")
    m1 = _node_to_mass(i)
    m2 = _node_to_mass(j)
    angular_momentum = int(function_definition["l"])
    d = sp.Symbol(R"R_\mathrm{res}")
    return DefinedExpression(
        expression=BreitWigner(s, mass, width, m1, m2, angular_momentum, d),
        definitions={
            mass: function_definition["mass"],
            width: function_definition["width"],
            m1: function_definition["ma"],
            m2: function_definition["mb"],
            d: function_definition["d"],
        },
    )


CHAIN_20 = CHAIN_DEFINITIONS[20]
K892_BW = _formulate_breit_wigner(
    propagator=CHAIN_20["propagators"][0],
    function_definition=get_function_definition("K892_BW", MODEL_DEFINITION),
    resonance=name_to_latex(CHAIN_20["name"]),
)
Math(aslatex(K892_BW))

#### Multi-channel Breit-Wigner

In [ ]:
@unevaluated
class MultichannelBreitWigner(sp.Expr):
    s: Any
    mass: Any
    channels: list[ChannelArguments] = argument(sympify=False)

    def evaluate(self):
        s = self.s
        m0 = self.mass
        width = sum(channel.formulate_width(s, m0) for channel in self.channels)
        return BreitWigner(s, m0, width)

    def _latex_repr_(self, printer: LatexPrinter, *args) -> str:
        latex = R"\mathcal{R}^\mathrm{BW}_\mathrm{multi}\left("
        latex += printer._print(self.s) + "; "
        latex += ", ".join(printer._print(channel.width) for channel in self.channels)
        latex += R"\right)"
        return latex


@frozen
class ChannelArguments:
    width: Any
    m1: Any = 0
    m2: Any = 0
    angular_momentum: Any = 0
    meson_radius: Any = 1

    def __attrs_post_init__(self) -> None:
        for name, value in asdict(self).items():
            object.__setattr__(self, name, sp.sympify(value))

    def formulate_width(self, s: Any, m0: Any) -> sp.Expr:
        Γ0 = self.width
        m1 = self.m1
        m2 = self.m2
        L = self.angular_momentum
        R = self.meson_radius
        ff = FormFactor(s, m1, m2, L, R) ** 2
        return Γ0 * m0 / sp.sqrt(s) * ff


x, y, z = sp.symbols("x:z")
s, m0, Γ0, m1, m2, L, d = sp.symbols("s m0 Gamma0 m1 m2 L R")
channels = [
    ChannelArguments(
        sp.Symbol(f"Gamma{i}"),
        sp.Symbol(f"m_{{a,{i}}}"),
        sp.Symbol(f"m_{{b,{i}}}"),
        sp.Symbol(f"L{i}"),
        d,
    )
    for i in [1, 2]
]
exprs = [
    MultichannelBreitWigner(s, m0, channels),
    BreitWigner(s, m0, Γ0, m1, m2, L, d),
    BreitWigner(s, m0, Γ0),
    EnergyDependentWidth(s, m0, Γ0, m1, m2, L, d),
    FormFactor(s, m1, m2, L, d),
    P(s, m1, m2),
    Kallen(x, y, z),
]
Math(aslatex({e: e.doit(deep=False) for e in exprs}))

In [ ]:
CHAIN_DEFINITIONS[0]

In [ ]:
get_function_definition("L1405_Flatte", MODEL_DEFINITION)

In [ ]:
def _formulate_multichannel_breit_wigner(
    propagator: dict, function_definition: dict, resonance: str, **kwargs
) -> DefinedExpression:
    channel_definitions = function_definition["channels"]
    if len(channel_definitions) < 2:
        msg = "Need at least two channels for a multi-channel Breit-Wigner"
        raise NotImplementedError(msg)
    node = propagator["node"]
    i, j = node
    s = _node_to_mandelstam(node)
    resonance_latex = name_to_latex(resonance)
    mass = sp.Symbol(f"m_{{{resonance}}}")
    width = sp.Symbol(Rf"\Gamma_{{{resonance}}}")
    m1 = _node_to_mass(i)
    m2 = _node_to_mass(j)
    angular_momentum = int(channel_definitions[0]["l"])
    d = sp.Symbol(f"R_{{{resonance_latex}}}")
    channels = [ChannelArguments(width, m1, m2, angular_momentum, d)]
    parameter_defaults = {
        mass: function_definition["mass"],
        width: channel_definitions[0]["gsq"],
        m1: channel_definitions[0]["ma"],
        m2: channel_definitions[0]["mb"],
        d: channel_definitions[0]["d"],
    }
    for i, channel_definition in enumerate(channel_definitions[1:], 2):
        Γi = sp.Symbol(
            Rf"\Gamma_{{{resonance_latex}}}^\text{{ch. {i}}}", nonnegative=True
        )
        mi1 = sp.Symbol(f"m_{{a,{i}}}", nonnegative=True)
        mi2 = sp.Symbol(f"m_{{b,{i}}}", nonnegative=True)
        angular_momentum = int(channel_definition["l"])
        channels.append(ChannelArguments(Γi, mi1, mi2, angular_momentum, d))
        parameter_defaults.update({
            mi1: channel_definition["ma"],
            mi2: channel_definition["mb"],
        })
    return DefinedExpression(
        expression=MultichannelBreitWigner(s, mass, channels),
        definitions=parameter_defaults,
    )


CHAIN_0 = CHAIN_DEFINITIONS[0]
L1405_Flatte = _formulate_multichannel_breit_wigner(
    propagator=CHAIN_0["propagators"][0],
    function_definition=get_function_definition("L1405_Flatte", MODEL_DEFINITION),
    resonance=name_to_latex(CHAIN_0["name"]),
)
Math(aslatex(L1405_Flatte))

#### Breit-Wigner with exponential

In [ ]:
s, m0, Γ0, m1, m2, γ = sp.symbols("s m0 Gamma0 m1 m2 gamma")
expr = BuggBreitWigner(s, m0, Γ0, m1, m2, γ)
Math(aslatex({expr: expr.doit(deep=False)}))

In [ ]:
CHAIN_DEFINITIONS[18]

In [ ]:
get_function_definition("K700_BuggBW", MODEL_DEFINITION)

In [ ]:
def _formulate_bugg_breit_wigner(
    propagator: dict, function_definition: dict, resonance: str, model: dict, **kwargs
) -> DefinedExpression:
    node = propagator["node"]
    i, j = node
    s = _node_to_mandelstam(node)
    mass = sp.Symbol(f"m_{{{resonance}}}", nonnegative=True)
    width = sp.Symbol(Rf"\Gamma_{{{resonance}}}", nonnegative=True)
    γ = sp.Symbol(Rf"\gamma_{{{resonance}}}")
    m1 = _node_to_mass(i)
    m2 = _node_to_mass(j)
    final_state = _get_final_state(model)
    return DefinedExpression(
        expression=BuggBreitWigner(s, mass, width, m1, m2, γ),
        definitions={
            m0: function_definition["mass"],
            Γ0: function_definition["width"],
            m1: final_state[i].mass,
            m2: final_state[j].mass,
            γ: function_definition["slope"],
        },
    )


CHAIN_18 = CHAIN_DEFINITIONS[18]
K700_BuggBW = _formulate_bugg_breit_wigner(
    propagator=CHAIN_0["propagators"][0],
    function_definition=get_function_definition("K700_BuggBW", MODEL_DEFINITION),
    resonance=name_to_latex(CHAIN_0["name"]),
    model=MODEL_DEFINITION,
)
Math(aslatex(K700_BuggBW))

#### General propagator dynamics builder

In [ ]:
def formulate_dynamics(chain_definition: dict, model: dict) -> DefinedExpression:
    expr = DefinedExpression()
    for propagator in chain_definition["propagators"]:
        parametrization = propagator["parametrization"]
        function_definition = get_function_definition(parametrization, model)
        function_type = function_definition["type"]
        if function_type == "BreitWigner":
            dynamics_builder = _formulate_breit_wigner
        elif function_type == "MultichannelBreitWigner":
            dynamics_builder = _formulate_multichannel_breit_wigner
        elif function_type == "BreitWignerWidthExpLikeBugg":
            dynamics_builder = _formulate_bugg_breit_wigner
        else:
            msg = f"No dynamics implementation for function type {function_type!r}"
            raise NotImplementedError(msg)
        expr *= dynamics_builder(
            propagator,
            function_definition=get_function_definition(parametrization, model),
            resonance=name_to_latex(chain_definition["name"]),
            model=model,
        )
    return expr

In [ ]:
Math(aslatex(formulate_dynamics(CHAIN_DEFINITIONS[0], MODEL_DEFINITION)))

In [ ]:
Math(aslatex(formulate_dynamics(CHAIN_DEFINITIONS[18], MODEL_DEFINITION)))

In [ ]:
Math(aslatex(formulate_dynamics(CHAIN_DEFINITIONS[20], MODEL_DEFINITION)))

## Construct `AmplitudeModel`

### Unpolarized intensity

In [ ]:
Topology = list[Union[int, "Topology"]]


def get_reference_subsystem(model: dict) -> FinalStateID:
    topology = get_reference_topology(model)
    return get_spectator_id(topology)


def get_spectator_id(topology: Topology) -> FinalStateID:
    spectator_candidates = {i for i in topology if isinstance(i, int)}
    if len(spectator_candidates) != 1:
        msg = f"Reference topology {topology} seems not to be a three-body decay"
        raise ValueError(msg)
    return next(iter(spectator_candidates))


def get_reference_topology(model: dict) -> Topology:
    distribution_def = _get_distribution_def(model)
    return distribution_def["decay_description"]["reference_topology"]


get_reference_subsystem(MODEL_DEFINITION)

In [ ]:
def get_existing_subsystem_ids(model: dict) -> list[FinalStateID]:
    distribution_def = _get_distribution_def(model)
    chain_defs = distribution_def["decay_description"]["chains"]
    subsystem_ids = {get_spectator_id(c["topology"]) for c in chain_defs}
    return sorted(subsystem_ids)


get_existing_subsystem_ids(MODEL_DEFINITION)

In [ ]:
from ampform.sympy import PoolSum

from ampform_dpd import (
    _AlignmentWignerGenerator,  # noqa:PLC2701,RUF100
    _generate_amplitude_index_bases,  # noqa:PLC2701,RUF100
)
from ampform_dpd.spin import create_spin_range


def formulate_aligned_amplitude(
    model: dict,
    λ1: sp.Rational | sp.Symbol,
    λ2: sp.Rational | sp.Symbol,
    λ3: sp.Rational | sp.Symbol,
) -> tuple[PoolSum, dict[sp.Symbol, sp.Expr]]:
    reference_subsystem = get_reference_subsystem(model)
    wigner_generator = _AlignmentWignerGenerator(reference_subsystem)
    λ0 = sp.Symbol(R"\lambda_0", rational=True)
    _λ0, _λ1, _λ2, _λ3 = sp.symbols(R"\lambda_(0:4)^{\prime}", rational=True)
    distribution_def = _get_distribution_def(model)
    decay_description = distribution_def["decay_description"]
    states = _get_states(decay_description["kinematics"])
    j0, j1, j2, j3 = (states[i].spin for i in sorted(states))
    A = _generate_amplitude_index_bases()
    amp_expr = PoolSum(
        sum(
            A[k][_λ0, _λ1, _λ2, _λ3]
            * wigner_generator(j0, λ0, _λ0, rotated_state=0, aligned_subsystem=k)
            * wigner_generator(j1, _λ1, λ1, rotated_state=1, aligned_subsystem=k)
            * wigner_generator(j2, _λ2, λ2, rotated_state=2, aligned_subsystem=k)
            * wigner_generator(j3, _λ3, λ3, rotated_state=3, aligned_subsystem=k)
            for k in get_existing_subsystem_ids(model)
        ),
        (_λ0, create_spin_range(j0)),
        (_λ1, create_spin_range(j1)),
        (_λ2, create_spin_range(j2)),
        (_λ3, create_spin_range(j3)),
    )
    return amp_expr, wigner_generator.angle_definitions


λ1, λ2, λ3 = sp.symbols(R"\lambda_(1:4)", rational=True)
amplitude_expr, _ = formulate_aligned_amplitude(MODEL_DEFINITION, λ1, λ2, λ3)
amplitude_expr.cleanup()

### Amplitude for the decay chain

In [ ]:
CHAIN_DEFINITIONS[0]

In [ ]:
def _formulate_recoupling(
    chain_definition: dict, vertex_idx: int
) -> tuple[sp.Indexed, sp.Number]:
    vertex_definitions = chain_definition["vertices"]
    if len(vertex_definitions) != 2:
        msg = f"Not a three-body decay: there are {len(vertex_definitions)} vertices"
        raise ValueError(msg)
    if vertex_idx not in {0, 1}:
        msg = f"Vertex index out of range. Can either be 0 or 1, not {vertex_idx}."
        raise ValueError(msg)
    vertex = chain_definition["vertices"][vertex_idx]
    vertex_type = vertex["type"]
    if vertex_type in {"helicity", "parity"}:
        helicity_coupling = True
        interaction = None
    elif vertex_type == "ls":
        msg = "No implementation yet for LS-coupling models"
        raise NotImplementedError(msg)
    else:
        msg = f"No implementation for vertex of type {vertex_type!r}"
        raise NotImplementedError(msg)
    symbol = _create_coupling_symbol(
        helicity_coupling,
        resonance=Str(name_to_latex(chain_definition["name"])),
        helicities=tuple(sp.Rational(v) for v in vertex["helicities"]),
        interaction=interaction,
        typ="production" if vertex_idx == 0 else "decay",
    )
    value = sp.S.One
    _warn_once("No implementation yet for value of the helicity recoupling factor")
    return symbol, value


def _warn_once(msg: str) -> None:
    warn(msg, category=UserWarning)

In [ ]:
Math(aslatex(dict(_formulate_recoupling(CHAIN_DEFINITIONS[0], i) for i in range(2))))

In [ ]:
def _get_final_state_helicities(
    chain_definition: dict,
) -> dict[FinalStateID, sp.Rational]:
    vertices = chain_definition["vertices"]
    helicities: dict[FinalStateID, sp.Rational] = {}
    for v in vertices:
        for helicity, node in zip(v["helicities"], v["node"]):
            if not isinstance(node, int):
                continue
            helicities[node] = sp.Rational(helicity)
    return {i: helicities[i] for i in sorted(helicities)}


_get_final_state_helicities(CHAIN_DEFINITIONS[0])

In [ ]:
def _get_resonance_helicity(
    chain_definition: dict,
) -> tuple[tuple[FinalStateID, FinalStateID], sp.Rational]:
    vertices = chain_definition["vertices"]
    for vertex in vertices:
        node = vertex["node"]
        if all(isinstance(i, int) for i in node):
            continue
        helicities = vertex["helicities"]
        for helicity, sub_node in zip(helicities, node):
            if isinstance(sub_node, abc.Sequence) and len(sub_node) == 2:
                return tuple(sub_node), sp.Rational(helicity)
    msg = "Could not find a resonance node"
    raise ValueError(msg)


_get_resonance_helicity(CHAIN_DEFINITIONS[0])

In [ ]:
def _get_spectator_helicity(chain_definition: dict) -> tuple[int, sp.Rational]:
    vertices = chain_definition["vertices"]
    for vertex in vertices:
        node = vertex["node"]
        if all(isinstance(i, int) for i in node):
            continue
        helicities = vertex["helicities"]
        for helicity, state_id in zip(helicities, node):
            if isinstance(state_id, int):
                return state_id, sp.Rational(helicity)
    msg = "Could not find vertex that contains a spectator"
    raise ValueError(msg)


_get_spectator_helicity(CHAIN_DEFINITIONS[0])

In [ ]:
def _get_decay_product_helicities(
    chain_definition: dict,
) -> tuple[tuple[int, sp.Rational], tuple[int, sp.Rational]]:
    vertices = chain_definition["vertices"]
    for v in vertices:
        node = v["node"]
        if all(isinstance(i, int) for i in node):
            helicities = v["helicities"]
            return tuple((i, sp.Rational(λ)) for i, λ in zip(node, helicities))
    msg = "Could not fine a helicity for any resonance node"
    raise ValueError(msg)


_get_decay_product_helicities(CHAIN_DEFINITIONS[0])

In [ ]:
def _get_weight(chain_definition: dict) -> tuple[sp.Symbol, complex | float]:
    value: complex | float
    value = complex(str(chain_definition["weight"]).replace(" ", "").replace("i", "j"))
    if not value.imag:
        value = value.real
    resonance_latex = name_to_latex(chain_definition["name"])
    _, resonance_helicity = _get_resonance_helicity(chain_definition)
    c = sp.IndexedBase(f"c^{{{resonance_latex}[{resonance_helicity}]}}")
    λ1, λ2, λ3 = _get_final_state_helicities(chain_definition).values()
    symbol = c[λ1, λ2, λ3]
    return symbol, value


Math(aslatex(dict([_get_weight(CHAIN_DEFINITIONS[0])])))

In [ ]:
def formulate_chain_amplitude(model: dict, chain_idx: int) -> sp.Expr:
    chain_defs = _get_decay_chains(model)
    chain_definition = chain_defs[chain_idx]
    definitions = {}
    # -----------------------
    dynamics = formulate_dynamics(chain_definition, model)
    for vertex in chain_definition["vertices"]:
        dynamics *= formulate_form_factor(vertex, model)
    definitions.update(dynamics.definitions)
    # -----------------------
    weight, weight_val = _get_weight(chain_definition)
    h_prod, h_prod_val = _formulate_recoupling(chain_definition, vertex_idx=0)
    h_dec, h_dec_val = _formulate_recoupling(chain_definition, vertex_idx=1)
    definitions[weight] = weight_val
    definitions[h_prod] = h_prod_val
    definitions[h_dec] = h_dec_val
    # -----------------------
    (i, λi), (j, λj) = _get_decay_product_helicities(chain_definition)
    θij, θij_expr = formulate_scattering_angle(i, j)
    definitions[θij] = θij_expr
    jR = sp.Rational(chain_definition["propagators"][0]["spin"])
    _, λR = _get_resonance_helicity(chain_definition)
    # -----------------------
    A = _generate_amplitude_index_bases()
    subsystem_id = get_spectator_id(chain_definition["topology"])
    k, λk = _get_spectator_helicity(chain_definition)
    λ = {i: λi, j: λj, k: λk}
    amplitude_symbol = A[subsystem_id][λ[1], λ[2], λ[3]]
    definitions[amplitude_symbol] = (
        weight * h_prod * h_dec * Wigner.d(jR, λR, λi - λj, θij) * dynamics.expression
    )
    return definitions

In [ ]:
definitions = formulate_chain_amplitude(MODEL_DEFINITION, chain_idx=0)
Math(aslatex(definitions))

### Full amplitude model